# Forecast with Cold Start Items

Consider the situation where a set of related new items got introduced into the catalog, one needs to forecast the future values for those items. In such a situation, little to none demand history is available for those items. This scenario is coined as "cold-start problem." 

<img src="../../common/images/amazon_forecast.png">

# Introduction

In this notebook, we walk through the process of generating forecasts for cold start items. Notice that <b>only DeepAR+ supports cold start.</b> At a high level, the flow can be summarized as follows. 

1. We follow the complete process with non-cold start items as in the previous notebooks such as 
 * [4. Getting_started_with_DeepAR+.ipynb](https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/4.%20Getting_started_with_DeepAR%2B.ipynb) or 
 * [6. Incorporating_Related_Time_Series_dataset_to_your_Predictor.ipynb](https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/6.Incorporating_Related_Time_Series_dataset_to_your_Predictor.ipynb). 
 
 The major addition is that, *the item meta information for both the cold start and non-cold start items are imported in the system*. 

2. Create another dataset with the cold start items, and create forecasts for those items. Here for simplicity, we only use target time series only dataset, but related time series can be incorporated as well. 


# Table of Contents

* Step 0: [Setting up](#setup)
* Step 1: [Preparing the Datasets](#prepare)
* Step 2: [Importing the Data for Non-Cold Start Items](#import)
 * Step 2a: [Creating a Dataset Group](#create)
 * Step 2b: [Creating a Target Dataset](#target)
 * Step 2c: [Creating a Item Meta Information Dataset](#related)
 * Step 2d: [Update the Dataset Group](#update)
 * Step 2e: [Creating a Target Time Series Dataset Import Job](#targetImport)
 * Step 2f: [Creating a Item Meta Information Dataset Import Job](#relatedImport)
* Step 3: [Create the DeepAR+ Predictor](#algo)
* Step 4: [Create a Forecast for non-Cold Start Items](#forecast)

The above steps complete the model training with the non-Cold Start items. Now we ready to import the cold start items, and generate their forecats. 

* Step 5: [Create a Cold-Start Target Time Series Dataset Import Job](#coldStartImport)
* Step 6: [Create a Forecast for the cold start items](#createColdStart) 
* Step 7: [Querying the Forecasts](#query)
* Step 8: [Exporting the Forecasts](#export)
* Step 9: [Clearning up your Resources](#cleanup)

# Step 0:  First let us setup Amazon Forecast<a class="anchor" id="setup">

This section sets up the permissions and relevant endpoints.

In [ ]:
import sys
import os

import boto3
import pandas as pd
import matplotlib.pyplot as plt

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

plt.rcParams['figure.figsize'] = (15.0, 5.0)

Configure the S3 bucket name and region name for this lesson.

- If you don't have an S3 bucket, create it first on S3.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [ ]:
text_widget_bucket = util.create_text_widget( "bucketName", "input your S3 bucket name" )
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

In [ ]:
bucketName = text_widget_bucket.value
assert bucketName, "bucketName not set."

region = text_widget_region.value
assert region, "region not set."

In [ ]:
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecast_query = session.client(service_name='forecastquery')

In [ ]:
# Create the role to provide to Amazon Forecast.
role_name = "ForecastNotebookRole-ColdStart"
role_arn = util.get_or_create_iam_role( role_name = role_name )

# Step 1: Preparing the Datasets<a class="anchor" id="prepare">
    
    
Here we use a synthetic dataset based on [electricity]() dataset, which consists of the hourly time series for 370 households (with item id 0 to 369). 

In this hypothetical senario, our goal is to generate forecasts for 4 new customers with item id 370 to 373. 

In [ ]:
zipLocalFilePath = "data/test.csv.gz"
localFilePath = "data/test.csv"

util.extract_gz( zipLocalFilePath, localFilePath )

In [ ]:
tdf = pd.read_csv(zipLocalFilePath, dtype = object)
tdf.head()

In [ ]:
tdf['target_value'] = tdf['target_value'].astype('float')

Let us plot one time series first.

In [ ]:
tdf[tdf['item_id'] == 'client_1'][-24*7*2:]\
    .plot(x='timestamp', y='target_value', figsize=(15, 8)); 

Next, we use an item meta information dataset that contains the information for both the non-cold start items (client 0 to 369) and cold start items (client 370 to 373). We call this meta information "type" in this specific case. Only one categorical feature is used in this demo, but in practice one normally has multiple categorical features. 

Note that for cold start items where little to none demand history exists, the algorithm can only "transfer" information from the existing items to the new ones through the meta information. Therefore, having informative and high quality meta data is the key for a good cold-start forecast. 

In [ ]:
# this metadata contains the cold start items' metadata as well.
localItemMetaDataFilePath = "data/itemMetaData.csv"
imdf = pd.read_csv(localItemMetaDataFilePath, dtype = object)

imdf.tail()

And the following figure shows the histogram of the category "type."

In [ ]:
imdf['type'].value_counts().plot(kind='bar');

In [ ]:
s3 = session.client('s3')

In [ ]:
targetTimeseriesDatakey = "cold-start/test.csv"

s3.upload_file(Filename=localFilePath, Bucket = bucketName, Key = f"{targetTimeseriesDatakey}")

In [ ]:
itemMetaDatakey = "cold-start/itemMetaData.csv"

s3.upload_file(Filename=localItemMetaDataFilePath, Bucket = bucketName, Key = f"{itemMetaDatakey}")

In [ ]:
project = "coldstart_demo"

Below, we specify key input data and forecast parameters

In [ ]:
freq = "H"
forecast_horizon = 48
timestamp_format = "yyyy-MM-dd HH:mm:ss"
delimiter = ','

## Step 2a. Creating a Dataset Group<a class="anchor" id="create">
First let's create a dataset group and then update it later to add our datasets.

In [ ]:
dataset_group = f"{project}_grp"
dataset_arns = []
create_dataset_group_response = forecast.create_dataset_group(Domain="CUSTOM",
                                                          DatasetGroupName=dataset_group,
                                                          DatasetArns=dataset_arns)

In [ ]:
print(f'Creating dataset group {dataset_group}')

In [ ]:
dataset_group_arn = create_dataset_group_response['DatasetGroupArn']

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

## Step 2b. Creating a Target Dataset<a class="anchor" id="target">
In this example, we will define a target time series. This is a required dataset to use the service.

Below we specify the target time series name af_demo_ts_4.

In [ ]:
ts_dataset_name = f"{project}_ts"
print(ts_dataset_name)

Next, we specify the schema of our dataset below. Make sure the order of the attributes (columns) matches the raw 
data in the files. We follow the same three attribute format as the above example.

In [ ]:
ts_schema_val = [{"AttributeName": "timestamp", "AttributeType": "timestamp"},
                {"AttributeName": "target_value", "AttributeType": "float"},
                {"AttributeName": "item_id", "AttributeType": "string"}]
ts_schema = {"Attributes": ts_schema_val}

In [ ]:
print(f'Creating target dataset {ts_dataset_name}')

In [ ]:
response = forecast.create_dataset(Domain="CUSTOM",
                               DatasetType='TARGET_TIME_SERIES',
                               DatasetName=ts_dataset_name,
                               DataFrequency=freq,
                               Schema=ts_schema
                              )

In [ ]:
ts_dataset_arn = response['DatasetArn']

In [ ]:
forecast.describe_dataset(DatasetArn=ts_dataset_arn)

## Step 2c. Creating ItemMetaData Dataset<a class="anchor" id="related">
In this example, we will define a Item Metadata Dataset.

Specify the related time series name af_demo_rts_4.

In [ ]:
item_metadata_dataset_name = f"{project}_meta"
print(item_metadata_dataset_name)

Specify the schema of your dataset here. Make sure the order of columns matches the raw data files. We follow the same three column format as the above example.

In [ ]:
meta_schema_val = [{"AttributeName": "item_id", "AttributeType": "string"},
              {"AttributeName": "category", "AttributeType": "string"}]
meta_schema = {"Attributes": meta_schema_val}

In [ ]:
print(f'Creating related dataset {meta_schema}')

In [ ]:
response = forecast.create_dataset(Domain="CUSTOM",
                               DatasetType='ITEM_METADATA',
                               DatasetName=item_metadata_dataset_name,
                               Schema=meta_schema
                              )

In [ ]:
meta_dataset_arn = response['DatasetArn']

In [ ]:
forecast.describe_dataset(DatasetArn=meta_dataset_arn)

## Step 2d. Updating the dataset group with the datasets we created<a class="anchor" id="update">
You can have multiple datasets under the same dataset group. Update it with the datasets we created before.

In [ ]:
dataset_arns = []
dataset_arns.append(ts_dataset_arn)
dataset_arns.append(meta_dataset_arn)
forecast.update_dataset_group(DatasetGroupArn=dataset_group_arn, DatasetArns=dataset_arns)

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

## Step 2e. Creating a Target Time Series Dataset Import Job<a class="anchor" id="targetImport">

In [ ]:
ts_dataset_import_job_response = forecast.create_dataset_import_job(DatasetImportJobName=dataset_group+"_1",
                                                             DatasetArn=ts_dataset_arn,
                                                             DataSource= {
                                                                 "S3Config" : {
                                                                     "Path": f"s3://{bucketName}/{targetTimeseriesDatakey}",
                                                                     "RoleArn": role_arn
                                                                 } 
                                                             },
                                                             TimestampFormat=timestamp_format)

In [ ]:
ts_dataset_import_job_arn=ts_dataset_import_job_response['DatasetImportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))
assert status

## Step 2f. Creating a Item Meta Data Dataset Import Job<a class="anchor" id="relatedImport">

In [ ]:
meta_dataset_import_job_response = forecast.create_dataset_import_job(DatasetImportJobName=dataset_group,
                                                             DatasetArn=meta_dataset_arn,
                                                             DataSource= {
                                                                 "S3Config" : {
                                                                     "Path": f"s3://{bucketName}/{itemMetaDatakey}",
                                                                     "RoleArn": role_arn
                                                                 } 
                                                             })

In [ ]:
meta_dataset_import_job_arn=meta_dataset_import_job_response['DatasetImportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=meta_dataset_import_job_arn))
assert status

# Step 3. Create  Predictor with the datasets<a class="anchor" id="algo">

In [ ]:
algorithm_arn = 'arn:aws:forecast:::algorithm/'

In [ ]:
algorithm = 'Deep_AR_Plus'
algorithm_arn_deep_ar_plus = algorithm_arn + algorithm
predictor_name_deep_ar = f'{project}_{algorithm.lower()}'

In [ ]:
print(f'[{predictor_name_deep_ar}] Creating predictor {predictor_name_deep_ar} ...')

In [ ]:
create_predictor_response = forecast.create_predictor(PredictorName=predictor_name_deep_ar,
                                                  AlgorithmArn=algorithm_arn_deep_ar_plus,
                                                  ForecastHorizon=forecast_horizon,
                                                  PerformAutoML=False,
                                                  PerformHPO=False,
                                                  InputDataConfig= {"DatasetGroupArn": dataset_group_arn},
                                                  FeaturizationConfig= {"ForecastFrequency": freq}
                                                 )

In [ ]:
predictor_arn_deep_ar = create_predictor_response['PredictorArn']

In [ ]:
status = util.wait(lambda: forecast.describe_predictor(PredictorArn=predictor_arn_deep_ar))
assert status

In [ ]:
forecast.describe_predictor(PredictorArn=predictor_arn_deep_ar)

# Step 4. Creating a Forecast<a class="anchor" id="forecast">

Next we re-train with the full dataset, and create the forecast.

In [ ]:
print(f"Done fetching accuracy numbers. Creating forecaster for DeepAR+ ...")

In [ ]:
forecast_name_deep_ar = f'{project}_deep_ar_plus_1'

In [ ]:
create_forecast_response_deep_ar = forecast.create_forecast(ForecastName=forecast_name_deep_ar,
                                                        PredictorArn=predictor_arn_deep_ar)

In [ ]:
forecast_arn_deep_ar_1 = create_forecast_response_deep_ar['ForecastArn']

In [ ]:
status = util.wait(lambda: forecast.describe_forecast(ForecastArn=forecast_arn_deep_ar_1))
assert status

In [ ]:
forecast.describe_forecast(ForecastArn=forecast_arn_deep_ar_1)

# Step 5. Creating a Cold-Start Target Time Series Dataset Import Job<a class="anchor" id="coldStartImport">
    
Now we are ready to generate the forecasts for the cold start problem. Notice that there is a system constraint such that at least 5 rows needs to be present for each item. Therefore, for the item that has less than 5 observations, we fill in with NaNs. In the following example, both Client 370 and 372 have zero observation, i.e., pure cold-start problem, while the other two have 5 target values. 

In [ ]:
localColdStartDataFilePath = "data/coldStartTargetData.csv"
# validation logic for at least 5 items
cstdf = pd.read_csv(localColdStartDataFilePath, dtype = object)
cstdf.head(20)

In [ ]:
coldStartTargetTimeseriesDatakey = "cold-start/coldStartTargetData.csv"

In [ ]:
s3.upload_file(Filename=localColdStartDataFilePath, 
               Bucket = bucketName, 
               Key = f"{coldStartTargetTimeseriesDatakey}")

In [ ]:
ts_cold_start_dataset_import_job_response = forecast.create_dataset_import_job(DatasetImportJobName=dataset_group+"_2",
                                                             DatasetArn=ts_dataset_arn,
                                                             DataSource= {
                                                                 "S3Config" : {
                                                                     "Path": f"s3://{bucketName}/{coldStartTargetTimeseriesDatakey}",
                                                                     "RoleArn": role_arn
                                                                 } 
                                                             },
                                                             TimestampFormat=timestamp_format)

In [ ]:
ts_cold_start_dataset_import_job_arn = ts_cold_start_dataset_import_job_response['DatasetImportJobArn']

In [ ]:
forecast.describe_dataset_import_job(DatasetImportJobArn=ts_cold_start_dataset_import_job_arn)

In [ ]:
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ts_cold_start_dataset_import_job_arn))
assert status

# Step 6. Create a ColdStart Item Forecast<a class="anchor" id="createColdStart">
    
    
Now we are ready to create the forecasts for all cold start items.

In [ ]:
forecast_name_deep_ar_2 = f'{project}_deep_ar_plus_2'

In [ ]:
create_forecast_response_deep_ar_2 = forecast.create_forecast(ForecastName=forecast_name_deep_ar_2,
                                                        PredictorArn=predictor_arn_deep_ar)

In [ ]:
forecast_arn_deep_ar_2 = create_forecast_response_deep_ar_2['ForecastArn']

In [ ]:
status = util.wait(lambda: forecast.describe_forecast(ForecastArn=forecast_arn_deep_ar_2))
assert status

In [ ]:
forecast.describe_forecast(ForecastArn=forecast_arn_deep_ar_2)

# Step 7. Querying the ColdStart Item Forecast<a class="anchor" id="query">
    
Now we plot the forecast, where the first vertical line is the forecast start date.

In [ ]:
forecast_response_deep_2 = forecast_query.query_forecast(
    ForecastArn=forecast_arn_deep_ar_2,
    Filters={"item_id": "client_370"})


fcst = forecast_response_deep_2['Forecast']['Predictions']
time_stamp = list(map(lambda x: pd.to_datetime(x['Timestamp']), fcst['p10']))
p10_fcst = list(map(lambda x: x['Value'], fcst['p10']))
p50_fcst = list(map(lambda x: x['Value'], fcst['p50']))
p90_fcst = list(map(lambda x: x['Value'], fcst['p90']))

plt.figure(figsize=(15, 10))
plt.plot(time_stamp, p50_fcst)
plt.fill_between(time_stamp, p10_fcst, p90_fcst, alpha=0.2)
plt.title("DeepAR Forecast 2");

# Step 8. Exporting your Forecasts<a class="anchor" id="export">

In [ ]:
forecast_export_name_deep_ar = f'{project}_cold_start_forecast_export_deep_ar_plus'
forecast_export_name_deep_ar_path = f"s3://{bucketName}/{forecast_export_name_deep_ar}"

In [ ]:
create_forecast_export_response_deep_ar = forecast.create_forecast_export_job(ForecastExportJobName=forecast_export_name_deep_ar,
                                                        ForecastArn=forecast_arn_deep_ar_2,
                                                        Destination={
                                                            "S3Config" : {
                                                                "Path": forecast_export_name_deep_ar_path,
                                                                "RoleArn": role_arn
                                                            }
                                                        })
forecast_export_arn_deep_ar = create_forecast_export_response_deep_ar['ForecastExportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_forecast_export_job(ForecastExportJobArn = forecast_export_arn_deep_ar))
assert status

# Step 9. Cleaning up your Resources<a class="anchor" id="cleanup">

Once we have completed the above steps, we can start to cleanup the resources we created. All delete jobs, except for `delete_dataset_group` are asynchronous, so we have added the helpful `wait_till_delete` function. 
Resource Limits documented <a href="https://docs.aws.amazon.com/forecast/latest/dg/limits.html">here</a>. 

In [ ]:
# Delete forecast export
util.wait_till_delete(lambda: forecast.delete_forecast_export_job(ForecastExportJobArn = forecast_export_arn_deep_ar))

In [ ]:
# Delete forecast
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecast_arn_deep_ar_1))
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecast_arn_deep_ar_2))

In [ ]:
# Delete predictor
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictor_arn_deep_ar))

In [ ]:
# Delete the target time series and related time series dataset import jobs
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=meta_dataset_import_job_arn))
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ts_cold_start_dataset_import_job_arn))

In [ ]:
# Delete the target time series and related time series datasets
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=ts_dataset_arn))
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=meta_dataset_arn))

In [ ]:
# Delete dataset group
forecast.delete_dataset_group(DatasetGroupArn=dataset_group_arn)

In [ ]:
# Delete IAM role
util.delete_iam_role( role_name )